In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import random as rd
import matplotlib.pyplot as plt

In [ ]:
glob.glob(os.path.join(os.path.abspath(""),"*"))

In [ ]:
full = pd.read_csv("pokemon_202110171358.csv")
df_pokemons = pd.read_csv("pokemon_202110171358.csv")
df_trainers = pd.read_csv("trainers_202110171359.csv")

In [ ]:
# print(df_pokemons)
# print(list(df_pokemons.columns))

# Obliczenie zadawanego zniszczenia i dodanie kolumny"demage"
#
    # Wzór:
    # damage = ((((2*level/5)+2)+(A/D)/50)+2)*random*type
        # gdzie: level = p.pokelevel
        # A = attack lub spatk
        # D = defense lub spdef
        # random = random integer percentage between 0.85 and 1.00 (inclusive)
        # type = type effectiveness (edited) 
# Zastanowić się czy współczynnik "random" przenieść do wyliczenia przed walką (żeby uniknąć powtarzalnych wartość dla całego dataframe)
df_pokemons["damage_physical"] = round(((((2 * df_pokemons["pokelevel"])/5)+(df_pokemons["attack"]/df_pokemons["defense"])/50)+2)*(rd.randint(85, 100)/100)) # *type - zdefiniować
df_pokemons["damage_special"] = round(((((2 * df_pokemons["pokelevel"])/5)+(df_pokemons["spatk"]/df_pokemons["spdef"])/50)+2)*(rd.randint(85, 100)/100)) # *type - zdefiniować

# Wylosowanie n pokemonów do dalszej gry
df_pokemons = df_pokemons.sample(100)

# Wylosowanie 3 pokemonów dla ai (przeciwnik)
df_ai_pokemons = df_pokemons.sample(3)

# Reset indeksów dla wylosowanych baraja_pokemonów
df_pokemons = df_pokemons.reset_index(drop=True)

# Reset indeksów dla wylosowanych ai_pokemonów
df_ai_pokemons = df_ai_pokemons.reset_index(drop=True)

# Wybór pokemona do symulacji
wybor_pokemona = int(input("Wybierz baraja_pokemona wpisując liczbę od 0 do 99:\n"))
wybor_ai_pokemona = int(input("Wybierz ai_pokemona wpisując liczbę od 0 do 2:\n"))

# Przypisanie wybranego pokemona do zmiennej baraja_poke
baraja_poke = df_pokemons.iloc[wybor_pokemona , :]
ai_poke = df_ai_pokemons.iloc[wybor_ai_pokemona, :]

# Wyświetlenie nazwy pokemona i jego typów
print("Pokemon baraja:",baraja_poke[2],baraja_poke[4],baraja_poke[5])
print("Pokemon AI:",ai_poke[2],ai_poke[4],ai_poke[5])

df_ai_pokemons.head()
# print(df_pokemons.iloc[ wybor_pokemona , : ]) #wyświetla dane dla wybranego pokemona po liczbie

In [ ]:
# Wyszukanie współczynnika "type" do wzoru demage
# Funkcja ma zwrócić wartości współczynnika "type" jako float, który będzie wykorzystany do obliczenia "demage" i "special demage"
#      zmienne wymagane do fukncji: typ pokemona atakującego (baraja_poke) i broniącego się (ai_poke)

df_type = pd.read_excel(r"c:\\Users\\Bartek\\Desktop\\Pokemon\\baraja\\Pokemony\\poke_type.xlsx")
# df_type = df_type.rename_axis()
# index = ["Normal","Fighting","Flying","Poison","Ground","Rock","Bug","Ghost","Steel","Fire","Water","Grass","Electric","Psychc","Ice","Dragon","Dark","Fairy"]
#             columns=["Normal","Fighting","Flying","Poison","Ground","Rock","Bug","Ghost","Steel","Fire","Water","Grass","Electric","Psychc","Ice","Dragon","Dark","Fairy"])
# df_type = df_type.drop(['Unnamed: 0'], axis=1)

df_type = df_type.set_index("Unnamed: 0")
df_type = df_type.rename_axis("", axis="index")

#zmienne_1: nazwa i typy pokemona baraja.
baraja_pokename = baraja_poke[2]
baraja_poke_type1 = baraja_poke[4]
baraja_poke_type2 = baraja_poke[5]
print(f"Your Pokemon is {baraja_pokename} and it is {baraja_poke_type1} and {baraja_poke_type2} type.")
#zmienne_2: nazwa i typy pokemona ai - przeciwnik.
ai_pokename = ai_poke[2]
ai_poke_type1 = ai_poke[4]
ai_poke_type2 = ai_poke[5]
print(f"Opponent's Pokemon is {ai_pokename} and it is {ai_poke_type1} and {ai_poke_type2} type.")

type_factor = float(df_type.loc[baraja_poke_type1, ai_poke_type1])
print(type_factor, type(type_factor))

In [ ]:
df_type

In [ ]:
full["damage_physical"] = round(((((2 * full["pokelevel"])/5)+(full["attack"]/full["defense"])/50)+2)*(rd.randint(85, 100)/100)) # *type - zdefiniować
full["damage_special"] = round(((((2 * full["pokelevel"])/5)+(full["spatk"]/full["spdef"])/50)+2)*(rd.randint(85, 100)/100)) # *type - zdefiniować

In [ ]:
def opp():
    rd_find = rd.randint(1,len(full))
    rd_poke = full.loc[rd_find]
    name =  rd_poke["pokename"]
    typ = rd_poke["type1"]
    index = rd_find
    print(f"Twój przeciwnik to {name}.\nJego typ to {typ}. Numer indexu pokemona to {index}")
    if typ == "Fire" or typ == "Water" or typ == "Grass" or typ == "Electric" or typ == "Ice" or typ == "Psychic":
        damage = rd_poke["damage_physical"] * rd_poke["hp"]
        return f"Wynik to {damage}"
    else:
        damage = rd_poke["damage_special"] * rd_poke["hp"]
        return f"Wynik to {damage}"
    


print(opp())

# def ranking():


# print(ranking())

In [ ]:
full["result"] = full["damage_special"] * full["hp"]


s = int(input("Wpisz index pokemona "))
s_chosen = full.loc[s]["result"]

print(s_chosen)
greater = full[full["result"] >= s_chosen].count()
print(greater["result"])
lower = full[full["result"] < s_chosen].count()

win = round((greater["result"] / len(full)),2)
loose = round((1 - win),2)
print(f"Wygrasz z {win} przeciwników oraz przegrasz z {loose} przeciwników")

In [ ]:
# 1. STATYSTYKA
#     > zestawienie sumaryczne pokelevelow przez typy pokemonów uwzględniając liczebność,
#     > wyznaczenie prawdopodobieństwa wylosowania kart danego typu,
#     > dodanie demage albo special demage w zależności od typu pokemona:
#         >> przyjmujemy zakres demage - dwie kolumny: demage min (wartość random=0,85), demage max (wartość random=1,0)
#             >>> dla każdego pokemona, następnie liczymy średni demage dla danych typów
#             >>> zestawiamy z prawdopodobieństwem wylosowania danego typu
#             >>> dla każdego typu przedstawić demage w formie wykresu skrzynkowego (max, min i średnia)
#         >> "defense" do statystyki (wyliczenie we wzorze "demage") = 1 - jest to wartość pobierana od przeciwnika = obniża "demage" atakującego,
#         >> Rodzaj ataku w funkcji typu pokemona:
#             >>> physical attack: Fire, Water, Grass, Electric, Ice, Psychic, Steel
#             >>> special attack: Normal, Fighting, Flying, Ground, Rock, Bug, Poison, Ghost, Dragon, Dark
#     > zastanowić się jak uwzględnić HP (zdrowie) - odejmowanie po ataku. 
#  Rodzaj ataku w funkcji typu pokemona:
#    physical attack: Fire, Water, Grass, Electric, Ice, Psychic, Steel
#    special attack: Normal, Fighting, Flying, Ground, Rock, Bug, Poison, Ghost, Dragon, Dark

In [ ]:
# print(df_pokemons)
# print(list(df_pokemons.columns))

# Obliczenie zadawanego zniszczenia i dodanie kolumny"demage"
#
    # Wzór:
    # damage = ((((2*level/5)+2)+(A/D)/50)+2)*random*type
        # gdzie: level = p.pokelevel
        # A = attack lub spatk
        # D = defense lub spdef
        # random = random integer percentage between 0.85 and 1.00 (inclusive)
        # type = type effectiveness (edited) 
# Zastanowić się czy współczynnik "random" przenieść do wyliczenia przed walką (żeby uniknąć powtarzalnych wartość dla całego dataframe)
df_pokemons["damage_physical"] = round(((((2 * df_pokemons["pokelevel"])/5)+(df_pokemons["attack"]/df_pokemons["defense"])/50)+2)*(rd.randint(85, 100)/100)) # *type - zdefiniować
df_pokemons["damage_special"] = round(((((2 * df_pokemons["pokelevel"])/5)+(df_pokemons["spatk"]/df_pokemons["spdef"])/50)+2)*(rd.randint(85, 100)/100)) # *type - zdefiniować

# Utworzenie listy z indeksów pokemonów - do wykorzystania przy losowaniu random
# pokemons_indexes = list(df_pokemons.index.values)
# Wylosowanie n=10 pokemonów do dalszej gry
## >>>>>>> Wypisać numer i nazwę pokemona <<<<<<< np. osobne 
df_pokemons = df_pokemons.sample(100)
df_pokemons = df_pokemons.reset_index(drop=True) #nowe indeksy dla wylosowanych pokemonów

while True:
    wybor_pokemona = int(input("Wybierz Pokemona wpisując liczbę od 0 do 99:\n")) #prośba o wybór pokemona do symulacji
    if wybor_pokemona >= 0 and wybor_pokemona < 100:
        break
    else:
        print("Wprowadź liczbę z zakresu od 0 do 99, która reprezentuje losowo wybranego Pokemona:")
    
    
print(df_pokemons.iloc[wybor_pokemona , : ]) #wyświetla dane dla wybranego pokemona po liczbie

In [ ]:
# Intro do symulatora walki pokemonów
print("""
         ***************************************      `;-.          ___,
         * Witaj w symulatorze walki Pokemonów *        `.`\_...._/`.-"`
         ***************************************          \        /      ,
                                                          /()   () \    .' `-._ 
                     _                                   |)  .    ()\  /   _.'     
         _ __   ___ | | _____ _ __ ___   ___  _ __       \  -'-     ,; '. < 
        | '_ \ / _ \| |/ / _ \ '_ ` _ \ / _ \| '_ \       ;.__     ,;|   > \\
        | |_) | (_) |   <  __/ | | | | | (_) | | | |     / ,    / ,  |.-'.-'
        | .__/ \___/|_|\_\___|_| |_| |_|\___/|_| |_|    (_/    (_/ ,;|.<`
        |_|                                               \    ,     ;-`
                                                           >   \    /
                                                          (_,-'`> .'
                                                               (_,'
""")


In [ ]:
def pull60(): #funkcja mająca na celu wybór listy 60 randomowych pokemonów z których będziemy wubierać nasz zespół oraz przeciwnika
    poke_list = full
    newlista = poke_list.sample(n = 60).sort_index()
    newlista = pd.DataFrame(newlista)
    return newlista



In [225]:
final_60 = pull60()

final_60 = final_60.reset_index(drop=True)


our_chempions = []


while len(our_chempions) < 3:
    poke = int(input("wpisz liczbę od 0 do 59: "))
    if poke < 0 or poke > 59:
        print("Niepoprawna wartość. Proszę powtórz.")
    elif poke in our_chempions:
        print("Wartość jest już w liście. Proszę powtórz!")
    else:
        our_chempions.append(poke)
print(our_chempions)




[6, 9, 8]


In [219]:
final_60

,trainerID,place,pokename,pokelevel,type1,type2,hp,maxhp,attack,defense,spatk,spdef,speed
0,257,0,Skarmory,43,Steel,Flying,124,124,89,140,54,80,80
1,409,2,Slaking,41,Normal,None,188,188,150,101,97,73,101
3,460,1,Wartortle,42,Water,None,116,116,73,87,74,87,68
4,511,4,Shiftry,29,Grass,Dark,101,101,73,50,67,50,61
5,651,0,Zangoose,27,Normal,None,86,86,76,47,47,47,63
6,720,1,Starmie,46,Water,Psychic,127,127,90,99,113,99,127
7,830,1,Wailmer,32,Water,None,136,136,61,38,61,38,54
9,891,1,Jigglypuff,42,Normal,Fairy,163,163,57,36,57,41,36
10,1673,1,Timburr,21,Fighting,None,70,70,46,35,23,27,27
11,2506,2,Huntail,56,Water,None,147,147,141,142,130,109,83


In [233]:
opp_chempions = final_60.sample(3) #wybór trzech randomowych przeciwników
opp_chempions = opp_chempions.reset_index(drop=True)

In [232]:
pierwszy_wojownik = our_chempions[0]
drugi_wojownik = our_chempions[1]
trzeci_wojownik = our_chempions[2]
df_our_chempions = final_60.iloc[[pierwszy_wojownik,drugi_wojownik, trzeci_wojownik]]
df_our_chempions = df_our_chempions.reset_index(drop=True)
df_our_chempions

,trainerID,place,pokename,pokelevel,type1,type2,hp,maxhp,attack,defense,spatk,spdef,speed
0,1229,1,Magcargo,42,Fire,Rock,109,109,62,120,87,87,45
1,1553,3,Linoone,45,Normal,None,141,141,84,76,66,76,111
2,1457,5,Solrock,50,Rock,Psychic,148,148,118,108,78,88,93


In [ ]:
def wykres_atak():
    num = int(input())

    pn = full.iloc[num]["pokename"]
    pl = full[full["pokename"] == pn]["attack"].to_numpy()
    z = full[full["pokename"] == pn]
    pl.sort()

    atk = full.iloc[num]["attack"]

    a = full[full["pokename"] == pn]["attack"].sort_values().to_numpy()
    z = 1 #pierwszy 

    for g in a:
        if g < full.iloc[num]["attack"]:
            z += 1

    plt.plot(pl) #Jak wyglądają pokemony tej samej nazwy na osi
    plt.scatter(x=z, y= atk, c="r") #Gdzie dokładnie jest nasz pokemon
    plt.title("Attack - " + pn )
    plt.xlabel("Numer w rankingu")
    plt.ylabel("Atak")



In [ ]:
wykres_atak()

In [234]:
#gra
num_n_poke = int(input("Wpisz index swojego Pokemona od 0 do 2: "))
m_poke = df_our_chempions.iloc[num_n_poke]["pokename"]

num_p_pokemon = int(input("Wpisz index przeciwnika: "))
p_poke = opp_chempions.iloc[num_p_pokemon]["pokename"]

In [235]:
print(m_poke,"vs", p_poke)

Magcargo vs Jirachi


In [236]:
def licznik(start, later):
    n = (later / start) * 20
    z = 20 - n
    n = round(n)
    z = round(z)


    tabela = f"{start} |" + "+" * int(n) + '-' * int(z) + f"| {later}"

    return tabela


In [ ]:
round(((((2 * df_pokemons["pokelevel"])/5)+(df_pokemons["attack"]/df_pokemons["defense"])/50)+2) * typee * randomowo

In [237]:
def attacking_opp(x, y):
    print()
    typ_atak = full.iloc[x]["type1"]
    typ_obrona = full.iloc[y]["type1"]
    type_factor = float(df_type.loc[typ_atak, typ_obrona])
    
    global life_opp_pokemon
    life_opp_pokemon = life_opp_pokemon - ((((2 * full.iloc[x]["pokelevel"])/5)+(full.iloc[x]["attack"]/full.iloc[y]["defense"])/50)+2) * ((rd.randint(85,100)) / 100) * type_factor
    life_opp_pokemon = round(life_opp_pokemon) 
    print(full.iloc[x]["pokename"], "atakuje", full.iloc[y]["pokename"])
    return "HP line:"
    

In [238]:
def attacking_us(x, y):
    print()
    typ_atak = full.iloc[x]["type1"]
    typ_obrona = full.iloc[y]["type1"]
    type_factor = float(df_type.loc[typ_atak, typ_obrona])
    
    global life_my_pokemon
    life_my_pokemon = life_my_pokemon - ((((2 * full.iloc[x]["pokelevel"])/5)+(full.iloc[x]["attack"]/full.iloc[y]["defense"])/50)+2) * ((rd.randint(85,100)) / 100) * type_factor
    life_my_pokemon = round(life_my_pokemon)
    print(full.iloc[x]["pokename"], "atakuje", full.iloc[y]["pokename"])
    return "HP line:"
    

In [240]:

player = 0
win_n_poke = []
win_opp_poke = []
start_my_poke = full.iloc[num_n_poke]["hp"] #życie początkowe naszego pokemona, ma być stałą wartością
start_opp_poke = full.iloc[num_p_pokemon]["hp"] #życie początkowe przeciwnika, ma być stałą wartością

life_my_pokemon = full.iloc[num_n_poke]["hp"] #życie naszego pokemona, ma być zmienne, odejmowane po ataku
life_opp_pokemon = full.iloc[num_p_pokemon]["hp"] #życie przeciwnika, ma być zmienne, odejmowane po ataku

while True:
    a = full.iloc[num_p_pokemon]["pokename"]
    b = full.iloc[num_n_poke]["pokename"]
    
    if player == 0:
        print(attacking_opp(num_n_poke, num_p_pokemon)) 
        print(licznik(start_opp_poke, life_opp_pokemon))

        if life_opp_pokemon <= 0:
            print(f"Pokemon {a} przegrał")
            win_n_poke.append(b)
            break
        else:
            player += 1
    
    elif player == 1:
        print(attacking_us(num_p_pokemon, num_n_poke))
        print(licznik(start_my_poke, life_my_pokemon))

        if life_my_pokemon <= 0:
            print(f"Pokemon {b} przegrał")
            win_opp_poke.append(a)
            break
        else:
            player -= 1


Smeargle atakuje Smeargle
HP line:
83 |++++++++++++++------| 60

Smeargle atakuje Smeargle
HP line:
157 |++++++++++++++++++--| 144

Smeargle atakuje Smeargle
HP line:
83 |++++++++------------| 35

Smeargle atakuje Smeargle
HP line:
157 |+++++++++++++++++---| 132

Smeargle atakuje Smeargle
HP line:
83 |++------------------| 9

Smeargle atakuje Smeargle
HP line:
157 |+++++++++++++++-----| 120

Smeargle atakuje Smeargle
HP line:
83 |-----------------------| -14
Pokemon Smeargle przegrał
